In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Daily Challenge**

Findings:
The analysis reveals a strong negative correlation between discounts above 40% and profitability, particularly in the "Furniture" category, which becomes consistently unprofitable when discounts exceed 60%. Sales volumes generally peak in November and December across all product categories. Additionally, California stands out as the top-performing state in terms of total sales, far surpassing others.

Recommendations:
It is advisable to cap discounts between 20% and 30%, especially during the first half of the year. For the last quarter (October to December), reducing discounts to a range of 10% to 20% is recommended to protect profitability. Sales efforts should also prioritize high-performing regions such as California and New York to maximize returns.

## **Phase 1: Data Scoping and Preparation**



### **1.   Data Ingestion and Initial Assessment:**




In [ ]:
import pandas as pd

superstore_file = pd.read_excel("/content/drive/MyDrive/GENAI/Week2/Day5/US Superstore data.xls")
df_superstore = pd.DataFrame(superstore_file)
df_superstore.head()

In [ ]:
df_superstore.shape[0]

In [ ]:
df_superstore.info()

In [ ]:
df_superstore.describe()

In [ ]:
df_superstore.isnull().sum()



### **2.   Data Cleaning and Preprocessing:**



In [ ]:
df_superstore.drop_duplicates(inplace=True)

On supprime les valeurs extremes des colonnes qui comportent des données quantitatives


In [ ]:
columns = df_superstore.select_dtypes(include='number').columns

outliers = pd.DataFrame()

for col in columns:
  Q1 = df_superstore[col].quantile(0.25)
  Q3 = df_superstore[col].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outlier = df_superstore[(df_superstore[col] < lower_bound) | (df_superstore[col] > upper_bound)]
  outlier['outliers'] = col
  outliers = pd.concat([outliers, outlier])

outliers.drop_duplicates(inplace=True)
outliers

In [ ]:
df_superstore.drop(outliers.index, inplace = True)
df_superstore.head()

In [ ]:
df_superstore.shape[0]

In [ ]:
df_superstore_cleaned = df_superstore.copy()

In [ ]:
df_superstore_cleaned.info()

On supprime ces colonnes car elles ne nous partagent pas d'informations pertinentes qui pourraient nous etre utile pour la suite


In [ ]:
df_superstore_reduced = df_superstore_cleaned.drop(["Order ID", "Customer ID", "Product ID"], axis = 1)
df_superstore_reduced.head()

In [ ]:
df_superstore_reduced.info()

In [ ]:
df_superstore_reduced2 = df_superstore_reduced.copy()

On utilise le one hot encoder pour les colonnes qui ont un faible nombre de cardinalitées pour eviter d'ajouter un trop grand nombre de colonnes. Sinon on utilise le label encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_superstore_reduced2["Customer Name"] = le.fit_transform(df_superstore_reduced2["Customer Name"])
df_superstore_reduced2["City"] = le.fit_transform(df_superstore_reduced2["City"])
df_superstore_reduced2["State"] = le.fit_transform(df_superstore_reduced2["State"])
df_superstore_reduced2["Sub-Category"] = le.fit_transform(df_superstore_reduced2["Sub-Category"])
df_superstore_reduced2["Product Name"] = le.fit_transform(df_superstore_reduced2["Product Name"])
df_superstore_reduced2.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

Oh_shipping = pd.get_dummies(df_superstore_reduced2["Ship Mode"])
oh_country = pd.get_dummies(df_superstore_reduced2["Country"])
oh_region = pd.get_dummies(df_superstore_reduced2["Region"])
oh_category = pd.get_dummies(df_superstore_reduced2["Category"])

df_superstore_reduced2 = pd.concat([df_superstore_reduced2, Oh_shipping, oh_country, oh_region, oh_category], axis = 1)
df_superstore_reduced2.drop(["Ship Mode", "Country", "Region", "Category"], axis = 1, inplace = True)
df_superstore_reduced2.head()



### **3.   Feature Engineering:**




In [ ]:
df_superstore_reduced["Profit Margin"] = (df_superstore_reduced["Profit"] / df_superstore_reduced["Sales"]) * 100
df_superstore_reduced["Order Year"] = df_superstore_reduced["Order Date"].dt.year
df_superstore_reduced["Order Month"] = df_superstore_reduced["Order Date"].dt.month
df_superstore_reduced["Ship Year"] = df_superstore_reduced["Ship Date"].dt.year
df_superstore_reduced["Ship Month"] = df_superstore_reduced["Ship Date"].dt.month

df_superstore_reduced.head()

On normalise les colonnes qui contiennent des données quantitatives


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_superstore_reduced[["Sales", "Quantity", "Discount", "Profit", "Profit Margin"]] = scaler.fit_transform(df_superstore_reduced[["Sales", "Quantity", "Discount", "Profit", "Profit Margin"]])
df_superstore_reduced.head()

In [ ]:
df = df_superstore_reduced.drop(["Order Date", "Ship Date"], axis = 1)
df.head()

## **Phase 2: Exploratory Analysis with Matplotlib**



### **1.   Time-Series Trend Investigation:**



In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets


category_dropdown = widgets.Dropdown(
    options=df['Category'].unique(),
    description='Category:'
)

def update_chart(selected_category):

    filtered_df = df[df['Category'] == selected_category]

    grouped = filtered_df.groupby(['Order Year', 'Order Month'])['Sales'].sum().reset_index()
    pivot_df = grouped.pivot(index='Order Month', columns='Order Year', values='Sales')

    plt.figure(figsize=(10, 5))
    for year in pivot_df.columns:
        plt.plot(pivot_df[year], marker='o', label=str(year))

    plt.title('Total Sales per Month by Year')
    plt.xlabel('Month')
    plt.ylabel('Total Sales')
    plt.xticks(ticks=range(1, 13), labels=[
        'Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Jui',
        'Jui', 'Aou', 'Sep', 'Oct', 'Nov', 'Dec'
    ])
    plt.legend(title='Year')
    plt.grid(True)
    plt.show()


widgets.interactive(update_chart, selected_category=category_dropdown)

In general, sales increase throughout the year
and are highest between November and December, regardless of product type.


### **2.   Geographic Performance Analysis:**



In [ ]:
interactive_tool = widgets.IntSlider(
    value = df["State"].nunique(),
    min = 0,
    max = df["State"].nunique(),
    step=1,
    description = "State",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'

)

def update_chart(number_state):

    filtered_df = df.groupby("State")["Sales"].sum().sort_values(ascending=False).head(number_state)
    plt.figure(figsize=(10, 5))
    plt.bar(filtered_df.index, filtered_df.values)
    plt.xlabel("State")
    plt.ylabel("Total Sales")
    plt.title(f"Top {number_state} States by Total Sales")
    plt.xticks(rotation=90)

widgets.interactive(update_chart, number_state=interactive_tool)


## **Phase 3:**





### **1.   Product Profitability Report:**



In [ ]:
filtered_df = df.groupby("Product Name")["Sales"].sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 5))
plt.barh(filtered_df.index, filtered_df.values, height=0.8)
plt.xlabel("Product Name")
plt.ylabel("Total Sales")
plt.title("Top 10 most profitable products")
plt.xticks(rotation=90)
plt.show()


### **2.   Discount Strategy Analysis:**



In [ ]:
import seaborn as sns

filtered_df = df.groupby("Product Name")["Sales"].sum().sort_values(ascending=False).head(10)
filtered_df2 = df[df["Product Name"].isin(filtered_df.index)]
plt.figure(figsize=(10, 5))
sns.scatterplot(x = filtered_df2["Discount"], y = filtered_df2["Profit"], hue = filtered_df2["Category"])
sns.regplot(x=filtered_df2["Discount"], y=filtered_df2["Profit"], scatter=False)
plt.xlabel("Discount")
plt.ylabel("Profit")
plt.title("Top 10 most profitable products")
plt.xticks(rotation=90)
plt.show()

Higher discounts generally reduce profit, especially in Technology and Furniture. Office Supplies are less affected.

## **Phase 4:**

In this project, Seaborn proved more efficient for quickly generating plots. Matplotlib, while slower to produce  visuals, offered essential fine-grained control for customizing layouts, axis labels, and spacing. Seaborn accelerated tasks like multi-category scatterplots and regression analysis, whereas Matplotlib was crucial when precise adjustments were needed. From now on, I will use Seaborn for rapid, multi-variable visualisation and I will use Matplotlib for adjusting visuals.

Findings : Analysis reveals a strong negative correlation between discount above 40% and profitability. Particularly within the "Furniture" category, which becomes consistently unprofitable at 60% discount. It also reveals that product are generarly more selled in november and december and that for all type of product. And it also provide the information that California is the state that make the most sales by far.

Recommendations : Reccoment capping the discount somwhere beteen 20% and 30%. Especially for the earlier month of the year. and reduce the discount between 10% and 20% for the last trimestre of the year. And focus the sales in california and new york.